In [1]:
using DataFrames
using JuMP, Gurobi

In [2]:
# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7;

In [99]:
function GetC()

    W_ = 19

    C = zeros((D*(W_+1), D*(W_+1)))

    for t in 1:(W_*D)
        for t_ in 1:(W_*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [4]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0, Monday=0.0)
    for i=1:N
        for j=1:N
            for d=1:D
                if x[i,j,week,d] == 1.0
                        df[i, 1+d] = j
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [88]:
function Optimize4WeeksAndFreeze(Y_old, Win, Wend)

    # create model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 600)

    # dont print output
    set_silent(model)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    D = 7 # days

    print(1)

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = Win:Wend, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    print(2)

    # OBJECTIVE FUNCTION
    # put penalty on having y[i,j] = 1, otherwise formulation put all of them to 1
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D) + sum(y[i, j] for i = 1:N, j = 1:N))

    print(3)

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d] - 1)

    print(4)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = Win:Wend, d = 1:D], x[i, i, w, d] == 0)

    print(5)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    print(6)

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = Win:Wend], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    print(7)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in Win:Wend, d in 1:D) == Wend - Win + 1)

    print(8)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in Win:Wend, d in 1:D) <= 1)

    print(9)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = Win:Wend))

    print(9.5)

    @constraint(model, [i = 1:N, j = 1:N], y[i, j] == y[j, i])
    
    print(10)

    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in Win:Wend, d in 1:D) <= 1 - Y_old[i, j])

    print(11)

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:3], x[i, j, w, d] == 0)

    print(12)

    # OPTIMIZE
    optimize!(model)

    print(13)

    # update old X, Y before returning them
    Y_old = Y_old .+ value.(y)

    print(14)

    return value.(x), Y_old

end


Optimize4WeeksAndFreeze (generic function with 2 methods)

In [94]:
Win_list = [1, 5, 9, 13, 17]
Wend_list = [4, 8, 12, 16, 19]

# first time, Y_old initialized to empty
Y_old = zeros(20, 20)   # teams never played against each other before

# define calendar list, each time append the calendar week to 
calendar = []
;

In [ ]:
for (Win, Wend) in zip(Win_list, Wend_list)

    println("Initial week: ", Win, ", end week: ", Wend)
    println("#############################################")

    # get value of X and Y
    X, Y_old = Optimize4WeeksAndFreeze(Y_old, Win, Wend)

    for w in Win:Wend
        push!(calendar, calendar_week(X, w))
    end

end

In [105]:
for w in 1:19
    display(calendar[w])
end

,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,11.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,20.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,18.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,13.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,12.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0,17.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,15.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,14.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,19.0,0.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,8.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,16.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,12.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,10.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,18.0
6,6,0.0,0.0,0.0,0.0,0.0,15.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,20.0
8,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,13.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,3.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,20.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,10.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,8.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,7.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,11.0,0.0,0.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,2.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,13.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,5.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6,6,0.0,0.0,0.0,0.0,0.0,14.0,0.0
7,7,0.0,0.0,0.0,0.0,12.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,9.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,8.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,12.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,14.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,16.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,9.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,7.0,0.0
6,6,0.0,0.0,0.0,8.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,5.0,0.0
8,8,0.0,0.0,0.0,6.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,4.0,0.0,0.0


...

,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,9.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,10.0,0.0
3,3,0.0,0.0,0.0,0.0,4.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,3.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,11.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,13.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,19.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,17.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,1.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,10.0,0.0,0.0
2,2,0.0,0.0,0.0,12.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,11.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,14.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,17.0
6,6,0.0,0.0,0.0,7.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,6.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,16.0
9,9,0.0,0.0,0.0,15.0,0.0,0.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,6.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,19.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,14.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,9.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,10.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,7.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,19.0
2,2,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4,4,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,3.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,11.0,0.0
7,7,0.0,0.0,0.0,0.0,13.0,0.0,0.0
8,8,0.0,0.0,0.0,15.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,10.0,0.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,7.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,11.0
3,3,0.0,0.0,0.0,19.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,17.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,8.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,12.0
7,7,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,5.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,18.0


In [109]:
# in this matrix all elements except the diagonal should be 1
println(sum([Y_old[i,i] for i in 1:20]))

# all other elements should be 1 (400 elements in total, diagonal = 20 element is 0, sum should be 380)
println(sum(Y_old))

0.0
380.0
